# Differential gene expression baseline

## Motivation

We want to add an additional view and an additional baseline to our feature attribution selection by determining genes that define clusters. In other words, we want to find marker genes for clusters which ideally correspond to markers of cell types or other conditions.

We follow the marker gene selection recommendations of the best practices book which is the Wilcoxon test.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import scanpy as sc
import pandas as pd

## Norman19

1. Load data
2. Find cluster labels
3. Calculate Wilcoxon tests with the references "rest" and "control"
4. Try to save the output in the format genes x cell types where the actual value is the p-value

In [4]:
norman = sc.read(f"{datasets_path}/scgen_norman19.h5ad")

### Wilcoxon of perturbation vs all other perturbations

In [ ]:
reference = "rest"

In [5]:
sc.tl.rank_genes_groups(norman,
                        groupby="perturbation_name",
                        layer="counts",
                        reference="rest",
                        method="wilcoxon",
                        key_added=f"wilcoxon_vs_{reference}")

In [6]:
norman_rest_df = pd.DataFrame(index=list(norman.var_names),
                              columns=list(norman.obs.perturbation_name.cat.categories))

for perturbation in norman.obs.perturbation_name.cat.categories:
    perturbation_de_result = sc.get.rank_genes_groups_df(norman,
                                                         group=perturbation,
                                                         key=f"wilcoxon_vs_{reference}")
    
    norman_rest_df[perturbation] = perturbation_de_result.set_index("names").pvals

In [7]:
norman_rest_df.to_csv(f"task1_DE_{reference}.csv",
                      sep=",")

### Wilcoxon of perturbation vs control

In [11]:
reference = "control"

In [12]:
sc.tl.rank_genes_groups(norman,
                        groupby="perturbation_name",
                        layer="counts",
                        reference="rest",
                        method="wilcoxon",
                        key_added=f"wilcoxon_vs_{reference}")

In [13]:
norman_control_df = pd.DataFrame(index=list(norman.var_names),
                                 columns=list(norman.obs.perturbation_name.cat.categories))

for perturbation in norman.obs.perturbation_name.cat.categories:
    perturbation_de_result = sc.get.rank_genes_groups_df(norman,
                                                         group=perturbation,
                                                         key=f"wilcoxon_vs_{reference}")
    
    norman_control_df[perturbation] = perturbation_de_result.set_index("names").pvals

In [14]:
norman_control_df.to_csv(f"norman_DE_{reference}.csv",
                      sep=",")

## HLCA

In [3]:
hlca = sc.read("../../datasets/hlca.h5ad")

Subsample to reduce runtime for DEG calculation. Subsample all cell types with >7000 cells to 7000 cells. If <7000, take all cells for differential expression.

In [26]:
idxs = []
for ct, count in hlca.obs.scanvi_label.value_counts().items():
    if count > 7000:
        idxs += list(sc.pp.subsample(hlca[hlca.obs.scanvi_label == ct], n_obs=7000, copy=True).obs.index)
    else:
        idxs += list(hlca[hlca.obs.scanvi_label == ct].obs.index)

subset = hlca[idxs]

In [37]:
%%time
sc.tl.rank_genes_groups(subset,
                        groupby="scanvi_label",
                        layer="counts",
                        reference="rest",
                        method="wilcoxon",
                        key_added="wilcoxon_cell_types")

In [39]:
hlca_df = pd.DataFrame(index=list(subset.var_names),
                                 columns=list(subset.obs.scanvi_label.cat.categories))

for cell_type in subset.obs.scanvi_label.cat.categories:
    cell_type_de_result = sc.get.rank_genes_groups_df(subset,
                                                      group=cell_type,
                                                      key="wilcoxon_cell_types")
    
    hlca_df[cell_type] = cell_type_de_result.set_index("names").pvals

In [42]:
hlca_df.index.name = 'gene_symbols'
hlca_df.to_csv("../../outputs/differential_expression/task2_DE_cell_types.csv", sep=",")